In [1]:
import gc, cv2
import numpy as np
import pandas as pd
import tensorflow as tf

from keras import applications
from keras import optimizers
from keras.models import Sequential, Model
from keras.layers import Input, Dense, BatchNormalization, MaxPooling2D, Dropout, Flatten
from keras.preprocessing.image import ImageDataGenerator

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [8]:
import tensorflow
vgg_model = tensorflow.keras.applications.VGG19(weights = "imagenet", include_top=False, input_shape = (224, 224, 3))

for layer in vgg_model.layers[:-5]:
    layer.trainable=False
    
for layer in vgg_model.layers[1:4]:
    layer.trainable=True
    
input = Input(shape=(224, 224, 3),name = 'image_input')
output_vgg16_conv = vgg_model(input)

x = BatchNormalization()(output_vgg16_conv)
x = MaxPooling2D(pool_size=(2, 2), padding='same')(x)
x = Dropout(0.2)(x)

x = Flatten()(x)
x = Dense(4096, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.3)(x)

x = Dense(1024, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.3)(x)

x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.3)(x)

x = Dense(102, activation='softmax')(x)

model = Model(inputs=input, outputs=x)

model.summary()

80150528/80134624 [==============================] - 1s 0us/step
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 image_input (InputLayer)    [(None, 224, 224, 3)]     0         
                                                                 
 vgg19 (Functional)          (None, 7, 7, 512)         20024384  
                                                                 
 batch_normalization (BatchN  (None, 7, 7, 512)        2048      
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 4, 4, 512)        0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 4, 4, 512)         0         
                                                              

In [9]:
model.load_weights("/content/drive/My Drive/Colab Notebooks/102_Flowers/flowers_vgg19.h5")

In [10]:
# labels = (train_generator.class_indices)
# labels = dict((v,k) for k,v in labels.items())
labels=[]
for i in range(1,103):
  labels.append(str(i))
len(labels)
labels.sort()
labels

['1',
 '10',
 '100',
 '101',
 '102',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '17',
 '18',
 '19',
 '2',
 '20',
 '21',
 '22',
 '23',
 '24',
 '25',
 '26',
 '27',
 '28',
 '29',
 '3',
 '30',
 '31',
 '32',
 '33',
 '34',
 '35',
 '36',
 '37',
 '38',
 '39',
 '4',
 '40',
 '41',
 '42',
 '43',
 '44',
 '45',
 '46',
 '47',
 '48',
 '49',
 '5',
 '50',
 '51',
 '52',
 '53',
 '54',
 '55',
 '56',
 '57',
 '58',
 '59',
 '6',
 '60',
 '61',
 '62',
 '63',
 '64',
 '65',
 '66',
 '67',
 '68',
 '69',
 '7',
 '70',
 '71',
 '72',
 '73',
 '74',
 '75',
 '76',
 '77',
 '78',
 '79',
 '8',
 '80',
 '81',
 '82',
 '83',
 '84',
 '85',
 '86',
 '87',
 '88',
 '89',
 '9',
 '90',
 '91',
 '92',
 '93',
 '94',
 '95',
 '96',
 '97',
 '98',
 '99']

In [11]:
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
import numpy as np

def output(location, new_model):
    img = load_img(location, target_size = (224, 224, 3))
    img = img_to_array(img)
    img = img / 255
    img = np.expand_dims(img, axis=0)
    answer = new_model.predict(img)
    y_class = answer.argmax(axis = -1)
    label = labels[np.argmax(answer)]
    print('label is', label)
    
    top_3 = np.argsort(answer[0])[:-4:-1]
#     for i in range(3):
#         print(" ({:.3})".format(answer[0][top_3[i]]))
    
    y = " ".join(str(x) for x in y_class)
    y = int(y)
    res = labels[y]
    print("category", res)
#     print(" ({:.3})".format(answer[0][top_3[0]]))
    pred_prob = answer[0][top_3[0]]
    print(" ({:.3})".format(pred_prob))
    
#     print ("new ********************")   
#     proba = answer[0]
#     idxs = np.argsort(proba)[::-1][:3]
#     for (i, j) in enumerate(idxs):
# #         y = labels[i]        
#         print(labels[i])
#     print("new prob", proba)
    return float(pred_prob), label

In [12]:
location = "/content/drive/My Drive/Colab Notebooks/102_Flowers/flowers_classes_top/95/10010.jpg"
output(location, model)

label is 95
category 95
 (0.985)


(0.9852928519248962, '95')

**Original model VGG19**

In [19]:
import cv2
import os
from shutil import copyfile
from pathlib import Path
final_acc = 0
folder = "/content/drive/My Drive/Colab Notebooks/102_Flowers/data/"
# print(len(os.listdir(folder + "001.Black_footed_Albatross")))
imagesFileNames = []
for filename in os.listdir(folder):    
    correct_count_By_folder=0
    print( len(os.listdir(folder+str(filename)) ))
    for img in os.listdir(folder+filename):
        if output(folder+filename+"/"+img, model)[1] ==  filename:
            correct_count_By_folder+=1
    final_accu_by_class = correct_count_By_folder/len(os.listdir(folder+filename))
    print('correctly classified : ' ,filename, correct_count_By_folder)
    print('accuracy for ' , filename , 'is ' , final_accu_by_class) 
    final_acc += final_accu_by_class
    print('final_acc : ', final_acc/102) 

#images = load_images_from_folder(folder)


Streaming output truncated to the last 5000 lines.
label is 9
category 9
 (0.979)
label is 9
category 9
 (0.995)
label is 9
category 9
 (0.998)
label is 9
category 9
 (0.998)
label is 9
category 9
 (0.986)
label is 9
category 9
 (1.0)
label is 9
category 9
 (0.747)
label is 9
category 9
 (0.996)
label is 9
category 9
 (0.998)
label is 9
category 9
 (0.999)
label is 9
category 9
 (0.947)
label is 9
category 9
 (0.987)
label is 9
category 9
 (0.971)
label is 9
category 9
 (0.966)
label is 9
category 9
 (0.997)
label is 9
category 9
 (0.836)
label is 9
category 9
 (1.0)
label is 9
category 9
 (0.999)
label is 9
category 9
 (0.998)
label is 9
category 9
 (0.927)
label is 9
category 9
 (0.996)
label is 33
category 33
 (0.553)
label is 9
category 9
 (0.999)
label is 9
category 9
 (1.0)
label is 9
category 9
 (0.983)
label is 9
category 9
 (0.877)
label is 9
category 9
 (0.687)
label is 9
category 9
 (0.999)
label is 9
category 9
 (1.0)
label is 9
category 9
 (0.999)
label is 9
category 9
 (1

**Note: test is just an empty subfolder, just ignore it**

**CauSIM** 

In [17]:
import ast
a_file = open("/content/drive/My Drive/Colab Notebooks/102_Flowers/Filters_Resp_Flowers.txt", "r")
# a_file = open("Gibbon_panda.txt", "r")
# a_file = open("Gimagenet_Test_2.txt", "r")

string_without_line_breaks = ""
for line in a_file:
    stripped_line = line.rstrip()
    string_without_line_breaks += stripped_line

import re
dict_Classes = ast.literal_eval(re.search('({.+})', string_without_line_breaks).group(0))

# for key, value in dict_Classes.items():
#     print ('classId', key)
#     print ('value', value)
a_file.close()
#print(dict_Classes)

In [ ]:
def insert_intermediate_layer_in_keras(model, layer_id, new_layer):
    from keras.models import Model

    layers = [l for l in model.layers]

    x = layers[0].output
    for i in range(1, len(layers)):
        if i == layer_id:
            x = new_layer(x)
        x = layers[i](x)

    model = Model(inputs=layers[0].input, outputs=x) ### inputs instead of input in this version
    return model

In [ ]:
import cv2
import os
from shutil import copyfile
from pathlib import Path
import collections
from keras.layers import Lambda
from keras import backend as K
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D, MaxPool2D 

final_acc = 0
folder = "/content/drive/My Drive/Colab Notebooks/102_Flowers/data/"
imagesFileNames = []
class_i=0
for filename in os.listdir(folder):
    for key, value in dict_Classes.items():      
        if key == filename: 
          print('key found:', key)
          weights_dic_Conv2_13= {}
          weights_dic_Conv2_13.update(value)
          weights_dic_Conv2_13 = sorted (weights_dic_Conv2_13.items ())
          arryWeights_last_After = []
          od = collections.OrderedDict(weights_dic_Conv2_13)
          for i in range(len(od)):
              if weights_dic_Conv2_13[i][1][0] >0:
                  arryWeights_last_After.append(1) 
              else:
                  arryWeights_last_After.append(0)
          
          def custom_layer_last(tensor):
            # print ('tesnor values :', tensor[0][0])
            return tensor * arryWeights_last_After # working version
#     ##############################################################
          lambda_layer = Lambda (custom_layer_last, name="lambda_New")
          vgg_model = tensorflow.keras.applications.VGG19(weights = "imagenet", include_top=False, input_shape = (224, 224, 3))
          vgg_alpha_Model = insert_intermediate_layer_in_keras (vgg_model, 21, lambda_layer)
  #         vgg_alpha_Model.summary()
          for layer in vgg_model.layers[:-5]:
              layer.trainable=False

          for layer in vgg_model.layers[1:4]:
              layer.trainable=True
          
          input = Input(shape=(224, 224, 3),name = 'image_input')
          output_vgg16_conv = vgg_alpha_Model(input)
                  
                  
          x = BatchNormalization()(output_vgg16_conv)
          x = MaxPooling2D(pool_size=(2, 2), padding='same')(x)
          x = Dropout(0.2)(x)

          x = Flatten()(x)
          x = Dense(4096, activation='relu')(x)
          x = BatchNormalization()(x)
          x = Dropout(0.3)(x)

          x = Dense(1024, activation='relu')(x)
          x = BatchNormalization()(x)
          x = Dropout(0.3)(x)

          x = Dense(512, activation='relu')(x)
          x = BatchNormalization()(x)
          x = Dropout(0.3)(x)

          x = Dense(102, activation='softmax')(x)

          new_model = Model(inputs=input, outputs=x)
  #         new_model.layers[1] = insert_intermediate_layer_in_keras (new_model.layers[1], 21, lambda_layer)

          new_model.load_weights('/content/drive/My Drive/Colab Notebooks/102_Flowers/flowers_vgg19.h5')
          # new_model = build_model()
          tensorflow.keras.backend.clear_session() 

          K.clear_session ()
          break;
    correct_count_By_folder=0
    print( len(os.listdir(folder+str(filename)) ))
    for img in os.listdir(folder+filename):
        if output(folder+filename+"/"+img, new_model)[1] ==  filename:
            correct_count_By_folder+=1
    final_accu_by_class = correct_count_By_folder/len(os.listdir(folder+filename))
    print('correctly classified : ' ,filename, correct_count_By_folder)
    print('accuracy for ' , filename , 'is ' , final_accu_by_class)
    final_acc += final_accu_by_class
    print('final_acc : ', final_acc/102) 
    print('number of classes : ', class_i) 
#images = load_images_from_folder(folder)


Streaming output truncated to the last 5000 lines.
 (1.0)
label is 8
category 8
 (1.0)
label is 8
category 8
 (1.0)
label is 8
category 8
 (1.0)
label is 8
category 8
 (1.0)
label is 8
category 8
 (1.0)
label is 8
category 8
 (1.0)
label is 8
category 8
 (1.0)
label is 8
category 8
 (1.0)
label is 8
category 8
 (1.0)
label is 8
category 8
 (1.0)
label is 8
category 8
 (1.0)
label is 8
category 8
 (1.0)
label is 8
category 8
 (1.0)
label is 8
category 8
 (1.0)
label is 8
category 8
 (1.0)
label is 8
category 8
 (1.0)
label is 8
category 8
 (1.0)
label is 8
category 8
 (1.0)
label is 8
category 8
 (1.0)
label is 8
category 8
 (1.0)
label is 8
category 8
 (1.0)
label is 8
category 8
 (1.0)
label is 8
category 8
 (1.0)
label is 8
category 8
 (1.0)
label is 8
category 8
 (1.0)
label is 8
category 8
 (1.0)
label is 8
category 8
 (1.0)
label is 8
category 8
 (1.0)
label is 8
category 8
 (1.0)
label is 8
category 8
 (1.0)
label is 8
category 8
 (1.0)
label is 8
category 8
 (1.0)
label is 8
cat